**Link of Original Code - https://github.com/llSourcell/How_to_make_a_text_summarizer/blob/master/vocabulary-embedding.ipynb**

# Installing packages

In [ ]:
!pip install jsonlines
#!pip install cPickle

# Importing and opening .JSONL dataset

Dataset link : https://www.kaggle.com/hsankesara/medium-articles 

On kaggle I got this dataset in .csv format then:

1. I converted .csv to .json
2. Convert .json to .jsonl(.jl)
3. Convert .jl to .pkl

1. I converted the file from  .csv to .json  

Here is gdrive link to download the .jsonfile : https://drive.google.com/file/d/1Qpq18as3I7qR-tgB2q93FBcMCpd1MRed/view?usp=sharing

2. Converting JSON file to JSONL

In [ ]:
import json
import jsonlines

with open("../input/aarticle/articles.json", 'r') as f:
    json_data = json.load(f)
    
with open("../working/articles.jl", 'w') as outfile:
    for entry in json_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
with open('../working/articles.jl') as myfile:
    firstNLines=myfile.readlines()[0:1]
    print(firstNLines)

# Creating pickle file

1.Use the 'content / description / text' as the 'desc' and the 'title' as the 'head'.
2.Once you have the data ready save it in a python pickle file as a tuple: (heads, descs, keywords) were heads is a list of all the head strings, descs is a list of all the article strings in the same order and length as heads. I ignore the keywrods information so you can place None.

Code Link - https://github.com/KevinDanikowski/nlp-tensorflow-practice/blob/master/text_summarizer/create_pickle_file.py 

In [ ]:
import _pickle as pickle
import jsonlines

heads = []
descs = []
keywords = []

with jsonlines.open('../working/articles.jl','r') as reader:
    i = 0
    for obj in reader:
        if i < 1000:
            i += 1
            head = obj["title"]
            desc = obj["text"]
            heads.append(head)
            descs.append(desc)
            keywords.append(None)
        else:
            break
        
# print(heads, descs, keywords)
with open('../working/tokens.pkl', 'wb') as f:
    pickle.dump((heads,descs,keywords),f)


# Word Embedding

#Generate intial word embedding for headlines and description

#The embedding is limited to a fixed vocabulary size (vocab_size) but a vocabulary of all the words that appeared in the data is built.

In [ ]:
FN = 'vocabulary-embedding'

In [ ]:
seed=42

In [ ]:
vocab_size = 40000

In [ ]:
embedding_dim = 300

In [ ]:
lower = False # dont lower case the text

# Read tokenized headlines and descriptions

# Load Data

In [ ]:
import pickle as pickle
FN0 = 'tokens' # this is the name of the data file which I assume you already have
with open('../working/%s.pkl'%FN0, 'rb') as fp:
    heads, descs, keywords = pickle.load(fp) # keywords are not used in this project

In [ ]:
if lower:
    heads = [h.lower() for h in heads]

In [ ]:
if lower:
    descs = [h.lower() for h in descs]

# Headings tuple

# Articles description

In [ ]:
i=0
heads[i]

In [ ]:
descs[i]

In [ ]:
type(desc)
#type(head)

In [ ]:
print(keywords[i])

In [ ]:
len(heads),len(set(heads))

In [ ]:
len(desc),len(set(desc))

# Build Vocabulary :

# Tokenize text, return vocab in order of usage

Link - https://github.com/KevinDanikowski/nlp-tensorflow-practice/blob/master/text_summarizer/text_summarizer.py

In [ ]:
from collections import Counter
def get_vocab(combinedText):
    #  to get vocab and count in another way, 
    words = combinedText.split()
    vocab = [word for word, word_count in Counter(words).most_common()]
    return vocab,words

vocab,words = get_vocab(heads[i]+descs[i])

In [ ]:
print (vocab[:50])
print ('...', len(vocab))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(words[w] for w in vocab);
plt.gca().set_xscale("log", nonposx='clip')
plt.gca().set_yscale("log", nonposy='clip')
plt.title('word distribution in headlines and discription')
plt.xlabel('rank')
plt.ylabel('total appearances');
warnings.warn(self.msg_depr % (key, alt_key))

# Index words

In [ ]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

In [ ]:
from itertools import chain
def get_idx(vocab):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return vocab, word2idx, idx2word
vocab,word2idx, idx2word = get_idx(vocab)


# Word Embeddings

In [ ]:
word2idx = get_idx(vocab)
print(len(vocab))
print(len(word2idx))
print(len(idx2word))

# Read Glove

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install keras

#  GLOVE code

In [ ]:
! pip install -q tqdm flair
import numpy as np
from tqdm import tqdm
glove_name= ! wget "http://nlp.stanford.edu/data/glove.6B.zip" -O glove.6B.zip && unzip glove.6B.zip

word2vec = {}
with open('glove.6B.300d.txt','r', encoding='utf-8',errors='ignore') as f:
  for line in tqdm(f, total =400000):
    #glove_n_symbols = line.decode().split(b":")[1]
    glove_n_symbols = line.split()
    word = glove_n_symbols[0]
    #coefs = np.fromstring(glove_n_symbols, "f", sep=" ")
    #coefs = np.asarray(glove_n_symbols[1:], dtype='float32')
    coefs = np.asarray([float(val) for val in glove_n_symbols[1:]])
    word2vec[word] = coefs

print('Found %s word vectors.' % len(word2vec))
#http://nlp.stanford.edu/data/glove.42B.300d.zip

In [ ]:
type(word)
print(word)

In [ ]:
type(coefs)

In [ ]:
print(coefs)

In [ ]:
print(glove_n_symbols)

In [ ]:
#print(word2vec)

In [ ]:
#glove_n_symbols = get_ipython().getoutput(u'wc -l {glove_name}')
#glove_n_symbols = int(glove_n_symbols[0].split()[0])
#%debug
glove_n_symbols = int(float(glove_n_symbols[0].split()[0]))
glove_n_symbols

In [ ]:
#%debug
import numpy as np
glove_index_dict = {}
glove_embedding_weights = np.empty((glove_n_symbols, embedding_dim),axis=0)
globale_scale=.1
print(glove_embedding_weights)

with open(glove_name, 'r',encoding='utf-8',errors='ignore') as fp:
    i = 0
    for l in fp:
        l = l.strip().split()
        w = l[0]
        glove_index_dict[w] = i
        glove_embedding_weights[i,:] = map(float,l[1:])
        i += 1
glove_embedding_weights *= globale_scale   


In [ ]:
exit